In [1]:
import pandas as pd
import yfinance as yf
import time
import requests
from bs4 import BeautifulSoup

In [30]:
tickers2='spy gme ^gspc'
tickers='xly xlc xlp vdc xle icln xlf vfh xlv vht vnq schh qqq vgt gdx afk egpt iemg vxus ezu ieur esge spy'
ticker_list=['xly', 'xlc', 'xlp', 'vdc', 'xle', 'icln', 'xlf', 'vfh', 'xlv', 'vht', 'vnq', 'schh', 'qqq', 'vgt', 'gdx', 'afk', 'egpt', 'iemg', 'vxus', 'ezu', 'ieur', 'esge', 'spy']
ticker_list2=['spy', 'gme', '^gspc']

In [3]:
info_dict={}
for i in range(len(ticker_list)):
    time.sleep(3)
    page=requests.get(f'https://etfdb.com/etf/{ticker_list[i]}/#etf-ticker-profile')
    soup=BeautifulSoup(page.content, 'html.parser')
    info=soup.find(class_='col-sm-6 col-xs-12')
    end=[k for k in info.get_text().split('\n') if k!='' and k!= 'Vitals' and k!= 'Investment Themes'].index('Trading Data')
    data=[k for k in info.get_text().split('\n') if k!='' and k!= 'Vitals' and k!= 'Investment Themes'][:end]

    for j in range(0,end,2):
        if j==0:
            info_dict[ticker_list[i]]={}
        info_dict[ticker_list[i]][data[j]]=data[j+1]

In [4]:
#info_dict

In [5]:
df=pd.DataFrame.from_dict(info_dict, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Ticker'}, inplace=True)
df.Ticker=df.Ticker.str.upper()

In [25]:
data=yf.download(tickers, start='2017-01-01', end='2021-02-08', group_by='ticker')

[*********************100%***********************]  3 of 3 completed


In [35]:
data_dict={}
for i in ticker_list2:
    data_dict[i.upper()]=data[i.upper()].reset_index()

In [52]:
info_df=pd.DataFrame.from_dict(info_dict, orient='index').reset_index()
info_df.rename(columns={'index': 'Ticker',
                        'Expense Ratio': 'Expense_Ratio',
                        'ETF Home Page': 'ETF_Home_Page',
                        'Tax Form': 'Tax_Form',
                        'Index Tracked': 'Index_Tracked',
                        'Asset Class': 'Asset_Class',
                        'Asset Class Size': 'Asset_Class_Size',
                        'Asset Class Style': 'Asset_Class_Style',
                        'Sector (General)': 'Sector_General',
                        'Sector (Specific)': 'Sector_Specific',
                        'Region (General)': 'Region_General',
                        'Region (Specific)': 'Region_Specific'}, inplace=True)

In [36]:
l=[]
for i in data_dict.keys():
    df=data_dict[i]
    df['Ticker']=i
    col=df.columns.to_list()
    col=col[-1:]+col[:-1]
    df=df[col]
    l.append(df)

In [37]:
data_df=pd.concat(l)
data_df.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

In [103]:
df=pd.merge(info_df, data_df, how='inner')
df['Ticker']=df.Ticker.str.upper()

In [104]:
df1=df[df.Ticker=='SPY'].reset_index(drop=True)
df1=df1[['Ticker', 'Date', 'Open', 'Close', 'Adj_Close',  'Volume']]
df1.rename(columns={'Ticker': 'MKT_Proxy',
                   'Open': 'MKT_Proxy_Open',
                   'Close': 'MKT_Proxy_Close',
                   'Adj_Close': 'MKT_Proxy_Adj_Close',
                   'Volume': 'MKT_Proxy_Volume'}, inplace=True)

In [105]:
df=df[df.Ticker!='SPY'].reset_index(drop=True)

In [106]:
df=pd.merge(df, df1, how='left', left_on='Date', right_on='Date')

In [38]:
#data_df.to_csv(r'filepath.csv')
#df.to_csv(r'filepath.csv')